PLR with STLAB
==============

Automated pupillometers are the standard instruments for measuring the PLR. These handheld devices are aimed at a persons eye to deliver a light stimulus and use infrared video recording and internal algorithms to provide an instant readout of the PLR and its associated parameters. The [NeurOptics PLR-3000](https://neuroptics.com/plr-3000-hand-held/) is one such device. 

<img src="../img/plr3000.png" width="200"/> 

Our integrating sphere can't compete with an automated pupillometer in terms portability and ease of use, but it can function in the same way. This protocol shows how we can do this.


In [4]:
import pandas as pd
plr_3000 = pd.read_csv('../data/PLR-3000_spectra.csv', index_col='uW') 

In [3]:
import sys
sys.path.insert(0, '../')
from time import sleep

import numpy as np

from pyplr.pupil import PupilCore
from pyplr.stlab import SpectraTuneLab
from pyplr.utils import unpack_data_pandas
from pyplr.preproc import butterworth_series
from pyplr.plr import plr_parameters, plot_plr

# Some constants
SAMPLE_RATE = 120
BASELINE = 2
DURATION = 6
ONSET = BASELINE*SAMPLE_RATE

# Set up pupil core
p = PupilCore()

# Set up STLAB
d = SpectraTuneLab(password='83e47941d9e930f6')
d.load_video_file()

# Annotation to be sent when the light comes on
annotation = p.new_annotation('LIGHT_ON')
    
# Start LightStamper and PupilGrabber
lst_future = p.light_stamper(
    annotation, 
    threshold=15, 
    timeout=DURATION)
pgr_future = p.pupil_grabber(
    topic='pupil.1.3d', 
    seconds=BASELINE+DURATION+2)
    
# Wait a few seconds...
sleep(BASELINE)

# Present stimulus
d.play_video_file()

while lst_future.running() or pgr_future.running():
    print('Waiting for futures...')
    sleep(1)

# Retrieve and process pupil data
data = unpack_data_pandas(pgr_future.result())
data = butterworth_series(
    data, 
    filt_order=3, 
    cutoff_freq=4/(SAMPLE_RATE/2), 
    fields=['diameter_3d'])

# Light_stamper timestamp
ts = lst_future.result()[1]

# Find the closest timestamp in the pupil data
idx = (np.abs(ts - data.index)).argmin()

# Trim and plot
start = idx-BASELINE*SAMPLE_RATE
end = idx+DURATION*SAMPLE_RATE
data = data.iloc[start:end]
data.reset_index(inplace=True)
ax = data['diameter_3d'].plot()
ax.axvline(x=ONSET, color='r')
ax.set_xticks(range(0,len(data), SAMPLE_RATE+1))
ax.set_xticklabels(range(-BASELINE,DURATION))
ax.set_ylabel('Pupil size (mm)')
ax.set_xlabel('Time relative to onset (s)')

STLAB device setup complete...


TypeError: load_video_file() missing 1 required positional argument: 'fname'